# Data Engineering in Python with databolt  - Quickly Extract data from Excel Files (d6tlib/d6tstack)

Excel are very common because non-technical user like accessing and manipulating data in Excel. For data engineering and data science those Excel files are not easily read however, for example `dask` and `pyspark` don't read Excel files. 

** In this workbook we will demonstrate how to use d6tstack to quickly extract data from messy Excel files into clean CSV data.**

We will be covering the following use cases:
* Check tab consistency for across multiple files
* Exract tabs from multipe Excel files
* Exract all tabs from an Excel file
* Extract data given unstrcutured files
* Clean empty columns and rows

In [35]:
import d6tstack.convert_xls
from d6tstack.convert_xls import XLSSniffer
from d6tstack.utils import PrintLogger

import pandas as pd
import dask.dataframe as dd
import glob

## Get sample data

We've created some dummy sample data which you can download. 

In [ ]:
import urllib.request
cfg_fname_sample = 'test-data-xls.zip'
urllib.request.urlretrieve("https://github.com/d6t/d6tstack/raw/master/"+cfg_fname_sample, cfg_fname_sample)
import zipfile
zip_ref = zipfile.ZipFile(cfg_fname_sample, 'r')
zip_ref.extractall('.')
zip_ref.close()

## Use Case: Extract all sheets from a single file


In [10]:
c = d6tstack.convert_xls.XLStoCSVMultiSheet('test-data/excel_adv_data/sample-xls-case-multisheet.xlsx', 
                                            output_dir = 'test-data/output', logger=PrintLogger())
c.convert_all()

converting file: sample-xls-case-multisheet.xlsx | sheet:  ok
converting file: sample-xls-case-multisheet.xlsx | sheet:  ok


/mnt/data/dev/d6t-lib/d6tstack/d6tstack/convert_xls.py:72: UserWarning: File test-data/excel_adv_data/sample-xls-case-multisheet.xlsx exists, skipping
  warnings.warn('File %s exists, skipping' %fname)


['test-data/output/sample-xls-case-multisheet.xlsx-Sheet1.csv',
 'test-data/output/sample-xls-case-multisheet.xlsx-Sheet2.csv']

In [11]:
ddf = dd.read_csv('test-data/output/sample-xls-case-multisheet.xlsx-*.csv')
ddf.compute().head()

,date,ticker,data,xls_sheet
0,2018-01-01,AAP,-0.672460,Sheet1
1,2018-01-02,AAP,-0.359553,Sheet1
2,2018-01-03,AAP,-0.813146,Sheet1
3,2018-01-04,AAP,-1.726283,Sheet1
4,2018-01-05,AAP,0.177426,Sheet1


## Use Case: Extract a sheets from multiple files


### Checking if the sheet exists across all files

In [18]:
cfg_fnames = list(glob.glob('test-data/excel_adv_data/sample-xls-case-multifile*.xlsx'))

In [20]:
# finds sheets across all files
sniffer = XLSSniffer(cfg_fnames)

print('all files have same sheet count?', sniffer.all_same_count())
print('')
print('all files have same sheet names?', sniffer.all_same_names())
print('')
print('all files contain sheet?', sniffer.all_contain_sheetname('Sheet1'))
print('')
print('detailed dataframe')
print('')
print(sniffer.df_xls_sheets.reset_index(drop=True).head())


all files have same sheet count? True

all files have same sheet names? True

all files contain sheet? True

detailed dataframe

                         file_name sheets_count sheets_idx sheets_names
0  sample-xls-case-multifile1.xlsx            1        [0]     [Sheet1]
1  sample-xls-case-multifile2.xlsx            1        [0]     [Sheet1]


### Extracting to csv

In [23]:
d6tstack.convert_xls.XLStoCSVMultiFile(cfg_fnames,output_dir = 'test-data/output',
                                       cfg_xls_sheets_sel_mode='name_global',cfg_xls_sheets_sel='Sheet1',logger=PrintLogger()).convert_all()


converting file: sample-xls-case-multifile1.xlsx | sheet: Sheet1 ok
converting file: sample-xls-case-multifile2.xlsx | sheet: Sheet1 ok


['test-data/output/sample-xls-case-multifile1.xlsx-Sheet1.csv',
 'test-data/output/sample-xls-case-multifile2.xlsx-Sheet1.csv']

In [25]:
ddf = dd.read_csv('test-data/output/sample-xls-case-multifile1.xlsx-*.csv')
ddf.compute().head()


,date,ticker,data,xls_sheet
0,2018-01-01,AAP,-0.353994,Sheet1
1,2018-01-02,AAP,-1.374951,Sheet1
2,2018-01-03,AAP,-0.643618,Sheet1
3,2018-01-04,AAP,-2.223403,Sheet1
4,2018-01-05,AAP,0.625231,Sheet1


## Use Case: Extract a sheets from multiple files, with complex layout


### Checking if the sheet exists across all files

In [26]:
cfg_fnames = list(glob.glob('test-data/excel_adv_data/sample-xls-case-badlayout1*.xlsx'))
print(len(cfg_fnames))

1


In [36]:
pd.read_excel(cfg_fnames[0]).head()

,Unnamed: 0,date,ticker,data,xls_sheet
0,NaN,2018-01-01,AAP,-1.306527,Sheet1
1,NaN,2018-01-02,AAP,1.658131,Sheet1
2,NaN,2018-01-03,AAP,-0.118164,Sheet1
3,NaN,2018-01-04,AAP,-0.680178,Sheet1
4,NaN,2018-01-05,AAP,0.666383,Sheet1


In [38]:
d6tstack.convert_xls.read_excel_advanced(cfg_fnames[0],
                                   sheet_name='Sheet1', header_xls_range="B2:E2").head()

,date,ticker,data,xls_sheet
0,2018-01-01,AAP,-1.306527,Sheet1
1,2018-01-02,AAP,1.658131,Sheet1
2,2018-01-03,AAP,-0.118164,Sheet1
3,2018-01-04,AAP,-0.680178,Sheet1
4,2018-01-05,AAP,0.666383,Sheet1


In [39]:
c = d6tstack.convert_xls.XLStoCSVMultiSheet(cfg_fnames[0],output_dir = 'test-data/output',logger=PrintLogger())
c.convert_all(header_xls_range="B2:B2")


converting file: sample-xls-case-badlayout1.xlsx | sheet:  ok
converting file: sample-xls-case-badlayout1.xlsx | sheet:  ok


/mnt/data/dev/d6t-lib/d6tstack/d6tstack/convert_xls.py:72: UserWarning: File test-data/excel_adv_data/sample-xls-case-badlayout1.xlsx exists, skipping
  warnings.warn('File %s exists, skipping' %fname)


['test-data/output/sample-xls-case-badlayout1.xlsx-Sheet1.csv',
 'test-data/output/sample-xls-case-badlayout1.xlsx-Sheet2.csv']

In [40]:
ddf = dd.read_csv('test-data/output/sample-xls-case-badlayout1.xlsx-*.csv')
ddf.compute().head() # dask breaks! use d6tstack.combine_csv


ValueError: Length mismatch: Expected axis has 1 elements, new values have 4 elements

In [42]:
cfg_fnames = list(glob.glob('test-data/output/sample-xls-case-badlayout1.xlsx-*.csv'))
len(cfg_fnames)
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames, all_strings=True)
c.combine().head()


,date,ticker,data,xls_sheet,filename
0,2018-01-01,AAP,-1.306526851735317,Sheet1,sample-xls-case-badlayout1.xlsx-Sheet1.csv
1,2018-01-02,AAP,1.658130679618188,Sheet1,sample-xls-case-badlayout1.xlsx-Sheet1.csv
2,2018-01-03,AAP,-0.1181640451285698,Sheet1,sample-xls-case-badlayout1.xlsx-Sheet1.csv
3,2018-01-04,AAP,-0.6801782039968504,Sheet1,sample-xls-case-badlayout1.xlsx-Sheet1.csv
4,2018-01-05,AAP,0.6663830820319143,Sheet1,sample-xls-case-badlayout1.xlsx-Sheet1.csv
